# ipycanvas: Interactive Canvas

## https://github.com/martinRenou/ipycanvas

Interactive widgets library exposing the browser's Canvas API

- BSD 3-Clause Licensed

**Installation:**

```bash
conda install -c conda-forge ipycanvas
```

or

```bash
mamba install -c conda-forge ipycanvas
```

In [ ]:
from math import pi

import ipycanvas

In [ ]:
c = ipycanvas.Canvas(size=(200, 200))

c

In [ ]:
# Draw simple triangle shape
c.begin_path()
c.move_to(75, 50)
c.line_to(100, 75)
c.line_to(100, 25)
c.fill()

In [ ]:
c = ipycanvas.Canvas(stroke_style='red', size=(200, 200))

# Draw smiley face
c.begin_path()
c.arc(75, 75, 50, 0, pi * 2, True) # Outer circle
c.move_to(110, 75)
c.arc(75, 75, 35, 0, pi, False) # Mouth (clockwise)
c.move_to(65, 65)
c.arc(60, 65, 5, 0, pi * 2, True) # Left eye
c.move_to(95, 65)
c.arc(90, 65, 5, 0, pi * 2, True) # Right eye
c.stroke()

c

In [ ]:
c

In [ ]:
c = ipycanvas.Canvas(size=(200, 200))

# Draw bubble
c.begin_path()
c.move_to(75, 25)
c.quadratic_curve_to(25, 25, 25, 62.5)
c.quadratic_curve_to(25, 100, 50, 100)
c.quadratic_curve_to(50, 120, 30, 125)
c.quadratic_curve_to(60, 120, 65, 100)
c.quadratic_curve_to(125, 100, 125, 62.5)
c.quadratic_curve_to(125, 25, 75, 25)
c.stroke()

c

In [ ]:
def rounded_rect(c, x, y, width, height, radius):
    c.begin_path()
    c.move_to(x, y + radius)
    c.line_to(x, y + height - radius)
    c.arc_to(x, y + height, x + radius, y + height, radius)
    c.line_to(x + width - radius, y + height)
    c.arc_to(x + width, y + height, x + width, y + height - radius, radius)
    c.line_to(x + width, y + radius)
    c.arc_to(x + width, y, x + width - radius, y, radius)
    c.line_to(x + radius, y)
    c.arc_to(x, y, x, y + radius, radius)
    c.stroke()

In [ ]:
c = ipycanvas.Canvas(size=(150, 150))

rounded_rect(c, 12, 12, 150, 150, 15)
rounded_rect(c, 19, 19, 150, 150, 9)
rounded_rect(c, 53, 53, 49, 33, 10)
rounded_rect(c, 53, 119, 49, 16, 6)
rounded_rect(c, 135, 53, 49, 33, 10)
rounded_rect(c, 135, 119, 25, 49, 10)

c.begin_path()
c.arc(37, 37, 13, pi / 7, -pi / 7, False)
c.line_to(31, 37)
c.fill()

for i in range(8):
    c.fill_rect(51 + i * 16, 35, 4, 4)

for i in range(6):
    c.fill_rect(115, 51 + i * 16, 4, 4)

for i in range(8):
    c.fill_rect(51 + i * 16, 99, 4, 4)

c.begin_path()
c.move_to(83, 116)
c.line_to(83, 102)
c.bezier_curve_to(83, 94, 89, 88, 97, 88)
c.bezier_curve_to(105, 88, 111, 94, 111, 102)
c.line_to(111, 116)
c.line_to(106.333, 111.333)
c.line_to(101.666, 116)
c.line_to(97, 111.333)
c.line_to(92.333, 116)
c.line_to(87.666, 111.333)
c.line_to(83, 116)
c.fill()

c.fill_style = 'white'
c.begin_path()
c.move_to(91, 96)
c.bezier_curve_to(88, 96, 87, 99, 87, 101)
c.bezier_curve_to(87, 103, 88, 106, 91, 106)
c.bezier_curve_to(94, 106, 95, 103, 95, 101)
c.bezier_curve_to(95, 99, 94, 96, 91, 96)
c.move_to(103, 96)
c.bezier_curve_to(100, 96, 99, 99, 99, 101)
c.bezier_curve_to(99, 103, 100, 106, 103, 106)
c.bezier_curve_to(106, 106, 107, 103, 107, 101)
c.bezier_curve_to(107, 99, 106, 96, 103, 96)
c.fill()

c.fill_style = 'black'
c.begin_path()
c.arc(101, 102, 2, 0, pi * 2, True)
c.fill()

c.begin_path()
c.arc(89, 102, 2, 0, pi * 2, True)
c.fill()

c

# Interaction during ipycanvas Animations

It can be fun to modify animations as they run, but reacting to user input during a standard animation loop can be difficult. This notebook shows one way to get around this by having the animation loop run in a separate thread.

In [ ]:
import numpy as np
from time import sleep
from threading import Event, Thread
from ipycanvas import Canvas, hold_canvas
from ipywidgets import Label, HTML, Button, HBox

For this example, we will simulate a number of particles, re-drawing them all each frame with the `fill_rects` function. 

In [ ]:
#Create our particles
n_particles = 4100
x = np.array(np.random.rayleigh(250, n_particles))
y = np.array(np.random.rayleigh(250, n_particles))
size = np.random.randint(1, 3, n_particles)

In [ ]:
# Create the canvas
canvas = Canvas(width=500, height=200)

In [ ]:
# Some useful functions that will be used in our animation loop:

# Draw the particles to the canvas
def draw_particles():
    with hold_canvas(canvas):
        canvas.clear() # Clear the old animation step       
        canvas.fill_style = 'green' 
        canvas.fill_rects(x, y, size) # Draw the new frame 

# Calculate new locations for the particles
def update_particle_locations():
    global x, y
    x = (x+1)%500
    y = (y +np.cos(x/100))%200

The key goal here is to enable interactivity. In this case this is done by creating a function that adds some new particles around a location, and setting this function as a callback that will be called when we click on the canvas.

In [ ]:
def handle_mouse_down(xpos, ypos):
    global x, y
    x_new = np.array(np.random.rayleigh(30, 100))+xpos-15
    y_new = np.array(np.random.rayleigh(30, 100))+ypos-15
    x = np.concatenate([x[-4000:], x_new])
    y = np.concatenate([y[-4000:], y_new])
    size = np.random.randint(1, 3, len(x))
    draw_particles()

# Register mouse click callback
canvas.on_mouse_down(handle_mouse_down)

With the setup out of the way, we can draw the first frame to the canvas and display it. Try clicking on the canvas to see the `handle_mouse_down` function's effect.

In [ ]:
draw_particles() # Running once to draw the first frame
display(canvas)

We get 'interactivity' as hoped - every click results in some new particles being added at that location. However, the issue comes when we try to interact while animating. Run the following loop and try to click on the canvas as it runs:

In [ ]:
for i in range(200):
    update_particle_locations()
    draw_particles()
    sleep(0.02)

Nothing happens in response to clicks while the animation is running. It is only once the loop finishes that we see flash of activity as some of the callbacks are belatedly run all at once. Now run the following cell and try interacting again:

In [ ]:
stopped = Event()
def loop():
    while not stopped.wait(0.02): # 0.02 secs -> ~50fps
        update_particle_locations()
        draw_particles()
Thread(target=loop).start()  

And to stop the animation:

In [ ]:
stopped.set()

### Explanation

In the first animation attempt, everything is running in a single thread. This means we cannot run callbacks (or any other code for that matter) while the animation loop runs. To fix this, we create and run a new Thread to run the animation loop (`Thread(target=loop).start() `). This frees the main thread to do other things, such as handle interaction. 

An `Event` has `set` and `clear` methods that allow for safe communication between threads. In this case our `loop` function that is running in its own thread will terminate if we run `stopped.set()`, allowing us to stop the animation. 

# Worked Example

For this example, as before, we will be updating some particles and adding new ones on mouse clicks. We use perlin noise to determine particle direction. The following cell results in a `perlin` function that takes in an x, y coordinate and returns a value - see [here](https://johnowhitaker.github.io/days_of_code/Playing_with_Perlin.html) for a more readable implementation.

In [ ]:
# Quick and dirty perlin noise function
def dgg(ix, iy, x, y):
    random = 2920 * np.sin(ix * 21942 + iy * 171324 + 8912) * np.cos(ix * 23157 * iy * 217832 + 9758)
    return ((x - ix)*np.cos(random) + (y - iy)*np.sin(random));
def perlin(x,y):
    x0, y0 = np.array(x).astype(int),np.array(y).astype(int)
    n0, n1 = dgg(x0, y0, x, y), dgg((x0 + 1), y0, x, y)
    ix0 = (n1 - n0) * (x - x0) + n0
    n0, n1 = dgg(x0, (y0 + 1), x, y), dgg((x0 + 1), (y0 + 1), x, y)
    return (((n1 - n0) * (x - x0) + n0) - ix0) * (y-y0) + ix0
perlin(0.3, 0.2) # Given x and y coords it generates a value

Now, as before, we set up our initial particle locations, define functions to update their positions (this time based on the perlin function) and to draw the particles to the canvas, register a callback to handle mouse clicks and start an animation thread. In addition, we add buttons to stop the animation, to restart it and to randomize the particle locations. Notice that when the start button is clicked it checks that the thread is not already running - removing this check will allow you to start several threads, all updating the animation, which results in a higher and higher framerate.

In [ ]:
# Setting up the canvas
canvas2 = Canvas(width=500, height=500) 

#Some particles (as before)
n_particles = 4100
x = np.array(np.random.rayleigh(250, n_particles))
y = np.array(np.random.rayleigh(250, n_particles))
size = np.random.randint(1, 3, n_particles)


def handle_mouse_down(xpos, ypos):
    global x, y
    x_new = np.array(np.random.rayleigh(30, 100))+xpos-15
    y_new = np.array(np.random.rayleigh(30, 100))+ypos-15
    x = np.concatenate([x[-4000:], x_new])
    y = np.concatenate([y[-4000:], y_new])
    size = np.random.randint(1, 3, len(x))
    draw_particles()

# Register mouse click callback
canvas2.on_mouse_down(handle_mouse_down)

def draw_particles():
    with hold_canvas(canvas2):
        canvas2.clear() # Clear the old animation step       
        canvas2.fill_style = 'green' 
        canvas2.fill_rects(x, y, size) # Draw the new frame 
        
def update_particle_locations():
    global x, y
    angles = perlin(x/35, y/35) * 3
    x += np.sin(angles)*0.3
    y += np.cos(angles)*0.3

stopped = Event()
def loop():
    while not stopped.wait(0.02): # the first call is in `interval` secs
        update_particle_locations()
        draw_particles()   
Thread(target=loop).start() # Start it by default

start_btn = Button(description='Start')
def start(btn):
    if stopped.isSet():
        stopped.clear()
        Thread(target=loop).start() 
start_btn.on_click(start)

stop_btn = Button(description='Stop')
def stop(btn):
    if not stopped.isSet():
        stopped.set()
stop_btn.on_click(stop)

reset_btn = Button(description='Randomize Particles')
def reset(btn):
    global x, y, size
    x = np.array(np.random.rayleigh(250, n_particles))
    y = np.array(np.random.rayleigh(250, n_particles))
    size = np.random.randint(1, 3, n_particles)
    draw_particles()
reset_btn.on_click(reset)

display(canvas2, HBox([start_btn, stop_btn, reset_btn]))

# Conclusions

Using threads allows us to create interactive animations that don't block our main thread from executing. This notebook shows one possible way to achieve this - there are alternatives such as the advanced python scheduler or the use of threading.Timer as in [this StackOverflow answer](https://stackoverflow.com/a/13151299). The latter was used in [the notebook](https://johnowhitaker.github.io/days_of_code/Interaction_with_ipycanvas.html) from which this example is derived. Whatever the specifics of the implementation, the core idea is to offload the animation loop to its own thread in some way.